In [10]:
import pandas as pd

data = pd.read_csv('data.csv')
data = data.head()

In [11]:
data = data.values.tolist()
print(data)

[[40.297875899999994, -75.5812935, 'REINDEER CT & DEAD END;  NEW HANOVER; Station 332; 2015-12-10 @ 17:10:52;', 19525.0, 'EMS: BACK PAINS/INJURY', '2015-12-10 17:10:52', 'NEW HANOVER', 'REINDEER CT & DEAD END', 'EMS', 'BACK PAINS/INJURY', 'winter', 'evening', 'weekday'], [40.2580614, -75.26467990000002, 'BRIAR PATH & WHITEMARSH LN;  HATFIELD TOWNSHIP; Station 345; 2015-12-10 @ 17:29:21;', 19446.0, 'EMS: DIABETIC EMERGENCY', '2015-12-10 17:29:21', 'HATFIELD TOWNSHIP', 'BRIAR PATH & WHITEMARSH LN', 'EMS', 'DIABETIC EMERGENCY', 'winter', 'evening', 'weekday'], [40.121181799999995, -75.3519752, 'HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-Station:STA27;', 19401.0, 'Fire: GAS-ODOR/LEAK', '2015-12-10 14:39:21', 'NORRISTOWN', 'HAWS AVE', 'Fire', 'GAS-ODOR/LEAK', 'winter', 'afternoon', 'weekday'], [40.116153000000004, -75.343513, 'AIRY ST & SWEDE ST;  NORRISTOWN; Station 308A; 2015-12-10 @ 16:47:36;', 19401.0, 'EMS: CARDIAC EMERGENCY', '2015-12-10 16:47:36', 'NORRISTOWN', 'AIRY ST & SWEDE ST',

In [15]:
from collections import defaultdict
from itertools import chain, combinations

def count_support(data, candidates):
    counts = defaultdict(int)
    for transaction in data:
        for candidate in candidates:
            if candidate.issubset(transaction):
                counts[candidate] += 1
    return counts

def generate_candidates(Lk, k):
    candidates = []
    for i in range(len(Lk)):
        for j in range(i + 1, len(Lk)):
            union_set = Lk[i].union(Lk[j])
            if len(union_set) == k:
                candidates.append(union_set)
    return candidates

def generate_frequent_itemsets(data, min_support):
    items = list(set(chain(*data)))
    C1 = [frozenset([item]) for item in items]
    transactions = [set(transaction) for transaction in data]

    candidate_counts = count_support(transactions, C1)
    L = [item for item, count in candidate_counts.items() if count >= min_support]
    k = 2
    frequent_itemsets = L
    while L:
        candidates = generate_candidates(L, k)
        candidate_counts = count_support(transactions, candidates)
        L = [item for item, count in candidate_counts.items() if count >= min_support]
        frequent_itemsets.extend(L)
        k += 1

    return frequent_itemsets

def generate_rules(data, min_support, min_confidence):
    frequent_itemsets = generate_frequent_itemsets(data, min_support)
    transactions = [set(transaction) for transaction in data]
    candidate_counts = count_support(transactions, frequent_itemsets)
    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            for Lk_minus_1 in range(1, len(itemset)):
                for subset in combinations(itemset, Lk_minus_1):
                    remain = itemset.difference(subset)
                    support_itemset = candidate_counts.get(itemset, 0)
                    support_subset = candidate_counts.get(frozenset(subset), 0)
                    if (
                        support_itemset >= min_support 
                        and support_subset > 0 
                        and support_itemset / support_subset >= min_confidence
                    ):
                        print(f"Rule: {list(subset)} => {list(remain)} (Confidence: {support_itemset / support_subset})")


min_support = 3
min_confidence = 0.5
generate_rules(data, min_support, min_confidence)


Rule: ['winter'] => ['EMS'] (Confidence: 0.8)
Rule: ['EMS'] => ['winter'] (Confidence: 1.0)
Rule: ['EMS'] => ['weekday'] (Confidence: 1.0)
Rule: ['weekday'] => ['EMS'] (Confidence: 0.8)
Rule: ['winter'] => ['weekday'] (Confidence: 1.0)
Rule: ['weekday'] => ['winter'] (Confidence: 1.0)
Rule: ['afternoon'] => ['winter'] (Confidence: 1.0)
Rule: ['winter'] => ['afternoon'] (Confidence: 0.6)
Rule: ['afternoon'] => ['weekday'] (Confidence: 1.0)
Rule: ['weekday'] => ['afternoon'] (Confidence: 0.6)
Rule: ['winter'] => ['EMS', 'weekday'] (Confidence: 0.8)
Rule: ['EMS'] => ['winter', 'weekday'] (Confidence: 1.0)
Rule: ['weekday'] => ['winter', 'EMS'] (Confidence: 0.8)
Rule: ['winter', 'EMS'] => ['weekday'] (Confidence: 1.0)
Rule: ['winter', 'weekday'] => ['EMS'] (Confidence: 0.8)
Rule: ['EMS', 'weekday'] => ['winter'] (Confidence: 1.0)
Rule: ['afternoon'] => ['winter', 'weekday'] (Confidence: 1.0)
Rule: ['winter'] => ['afternoon', 'weekday'] (Confidence: 0.6)
Rule: ['weekday'] => ['afternoon', '